In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Importing AnimalShelter CRUD Python module into Milestone
import AnimalShelter
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Initializing username and password variables to access CRUD Python module name.
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo image variables
image_filename = 'GSlogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    
    # Code to add Grazioso Salvare's logo to the dashboard
    html.Center(
        html.A([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                     style={'height':'30%','width':'30%'},
                    )],
            href='https://www.snhu.edu') # image links to their home page
    ),
    html.Hr(),
        
    # Radio Buttons to filter options for the Data table
    html.Div(
        # Initalizing label and value variables for data table filters, Starts on Reset Filter
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label':'Water Rescue', 'value': 'water'},
                {'label':'Wilderness Rescue','value': 'wilderness'},
                {'label':'Disaster Rescue', 'value':'disaster'},
                {'label':'Reset', 'value':'reset'},
            ],
            value='Reset', 
            inline=True # Displays radio buttons horizontally
        )
    ),             
    html.Hr(),
    
    # Setup Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                ],
        data=df.to_dict('records'),
        
        # Data Table features
        editable=False,
        filter_action='native',
        sort_action='native',
        sort_mode='multi',
        column_selectable=False,
        # specified by milestone to ensure proper callback function
        row_selectable='single',
        row_deletable=False,
        selected_columns=[],
        # Ensures that the Geolocation Map will initialize to the first animal on the data table
        selected_rows=[0],
        page_action='native',
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
    
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                 html.Div(id='graph-id',
                          className='col s12 m6',
                         ),
                 html.Div(id='map-id',
                          className='col s12 m6',
                         )
             ]
            ),
    html.Hr(),
    
    # Unique identifier such as your name and "copyright" date
    html.Center(html.A(html.P('Coded by Raphael Coloma © 2023')))
])

#############################################
# Interaction Between Components / Controller
#############################################

# This callback will filter the data table based on passed radio filter value
@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns')],
              [Input('filter-type', 'value')])

def update_dashboard(filter_type):
    # establish df is not a new local variable
    global df
    # start case - Filter set to reset. Show all animal documents
    if filter_type == "reset":
        df = pd.DataFrame.from_records(db.read({}))
    # Water Rescue Filter
    elif filter_type == "water":
        df = pd.DataFrame.from_records(db.read({
            "animal_type":"Dog",
            "breed": {"$in":["Labrador Retriever Mix","Chesa Bay Retr Mix", "Newfoundland Mix"]},
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}
        }))
    # Mountain Rescue Filter
    elif filter_type == "wilderness":
        df = pd.DataFrame.from_records(db.read({
            "animal_type":"Dog",
            "breed": {"$in":["German Shepherd","Alaskan Malamute", "Old English Sheepdog","Siberian Husky","Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}
        }))
    # Disaster Rescue Filter
    elif filter_type == "disaster":
        df = pd.DataFrame.from_records(db.read({
            "animal_type":"Dog",
            "breed": {"$in":["German Shepherd","Doberman Pinsch", "Golden Retriever","Bloodhound","Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":20},
            "age_upon_outcome_in_weeks":{"$lte":300}
        }))
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    # Cleanup MongoDB _id field
    #df.drop(columns=['_id'], inplace=True)
    data=df.to_dict('records')
    return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(Output('graph-id', "children"),
              [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    # assign data variable
    df = pd.DataFrame.from_records(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "selected_rows")])
def update_map(viewData, derived_virtual_selected_rows):  
    if viewData is None:
        return
    elif derived_virtual_selected_rows is None:
        return
    
    # Because we only allow single row selection, the list can be converted to a row index here
    dff = pd.DataFrame.from_dict(viewData)
    if derived_virtual_selected_rows is None:
        index = selected_row
    else:
        index = derived_virtual_selected_rows[0]
        
    # gets and assigns Longitude and Latitude variables to center map on selected animal
    posLat = dff.iloc[index, 13]
    posLong = dff.iloc[index,14]
    
    # gets and assigns animal name to variable
    # if statement to assign null data as no name
    name = dff.iloc[index,9]
    if name == "":
        name = "No Name"
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[posLat, posLong], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Marker is positioned based on Longitude and Latitude variables
            # Column 4 defines the breed for the animal
            dl.Marker(position=[posLat,posLong], children=[
                dl.Tooltip(dff.iloc[index,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:11406/
